1. import libraries

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm
import plotly.express as px
import pickle
from dotenv import load_dotenv
import os

In [2]:
pd.set_option('display.max_columns', None)

2. import data from database by sql 

In [3]:
load_dotenv()

ENDPOINT = os.getenv('ENDPOINT')
PORT = os.getenv('PORT')
USERNAME = os.getenv('USERNAME')
PASSWORD = os.getenv('PASSWORD')
DBNAME = os.getenv('DBNAME')

In [4]:
database_conn = create_engine(f'mysql+pymysql://{USERNAME}:{PASSWORD}@{ENDPOINT}/{DBNAME}')

query = """
WITH 
temp_table1 AS (
    SELECT *
    FROM account
    LEFT JOIN account_usage USING (account_id)
    ),
temp_table2 AS (
	SELECT *
	FROM temp_table1
	LEFT JOIN customer USING (customer_id)
    ),
temp_table3 AS (
	SELECT *
	FROM temp_table2
	LEFT JOIN city USING (zip_code)
    )
SELECT * FROM temp_table3
LEFT JOIN churn_status USING (customer_id)
;

"""

final_analytic = pd.read_sql(query, database_conn)
final_analytic

,customer_id,zip_code,account_id,tenure_months,num_referrals,has_internet_service,internet_type,has_unlimited_data,has_phone_service,has_multiple_lines,has_premium_tech_support,has_online_security,has_online_backup,has_device_protection,contract_type,paperless_billing,payment_method,avg_long_distance_fee_monthly,total_long_distance_fee,avg_gb_download_monthly,stream_tv,stream_movie,stream_music,total_monthly_fee,total_charges_quarter,total_refunds,gender,age,senior_citizen,married,num_dependents,area_id,city,latitutde,longitude,population,status,churn_label,churn_category,churn_reason
0,0334-ZFJSR,92123,AAJU-HMJLK,55,0,Yes,Cable,No,Yes,Yes,Yes,Yes,Yes,No,One Year,Yes,Credit Card,35.38,1945.90,13,No,No,No,66.05,3462.10,44.53,Female,41,No,Yes,0,371,San Diego,32.808814,-117.134694,25232,Stayed,No,,
1,4006-HKYHO,93311,AAMB-TJYWC,63,0,Yes,DSL,Yes,Yes,No,Yes,Yes,Yes,Yes,Two Year,Yes,Bank Withdrawal,29.58,1863.54,47,Yes,Yes,Yes,86.70,5309.50,0.00,Male,24,No,No,0,696,Bakersfield,35.162070,-119.194488,20440,Stayed,No,,
2,3258-SYSWS,92040,AANL-MWPZF,72,0,Yes,Fiber Optic,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Two Year,No,Bank Withdrawal,24.67,1776.24,26,Yes,Yes,No,113.80,7845.80,0.00,Male,75,Yes,No,0,329,Lakeside,32.909873,-116.906774,42277,Stayed,No,,
3,1820-DJFPH,95555,ABBQ-EXMMW,72,4,No,None,No,Yes,Yes,No,No,No,No,Two Year,Yes,Bank Withdrawal,48.29,3476.88,0,No,No,No,24.05,1709.15,0.00,Female,59,No,Yes,3,1310,Orick,41.336354,-124.044354,494,Stayed,No,,
4,0582-AVCLN,93304,ABHW-EXJZF,38,0,No,None,No,Yes,No,No,No,No,No,Two Year,No,Credit Card,43.19,1641.22,0,No,No,No,20.30,743.05,0.00,Female,43,No,No,0,690,Bakersfield,35.339796,-119.023552,44588,Stayed,No,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6848-YLDFR,96035,ZWXF-QGPFD,58,9,Yes,DSL,Yes,Yes,No,Yes,No,Yes,No,One Year,Yes,Credit Card,35.35,2050.30,29,Yes,Yes,Yes,75.20,4300.80,0.00,Male,54,No,Yes,2,1546,Gerber,40.031940,-122.176023,3357,Stayed,No,,
7039,7587-AOVVU,95521,ZXNR-BOLMC,27,0,Yes,Fiber Optic,Yes,Yes,Yes,Yes,Yes,Yes,No,Month-to-Month,No,Bank Withdrawal,14.07,379.89,15,Yes,No,No,100.75,2793.55,0.00,Male,39,No,Yes,1,1289,Arcata,40.839958,-124.003757,19596,Stayed,No,,
7040,7479-NITWS,92341,ZXYG-EZZNH,7,0,Yes,Fiber Optic,No,Yes,Yes,No,No,No,Yes,Month-to-Month,Yes,Credit Card,36.84,257.88,17,Yes,No,No,89.35,631.85,0.00,Male,35,No,No,0,451,Green Valley Lake,34.244411,-117.072654,317,Churned,Yes,Price,Price too high
7041,8910-LEDAG,93651,ZYQO-YPQDV,34,2,Yes,Fiber Optic,Yes,Yes,No,No,No,No,No,Two Year,No,Credit Card,12.98,441.32,8,No,No,No,71.55,2427.35,0.00,Male,72,Yes,Yes,0,805,Prather,37.007238,-119.505661,1314,Stayed,No,,


3. data processing

In [5]:
df = final_analytic.copy()

In [6]:
df['churn_label'].unique()

array(['No', 'Yes', ''], dtype=object)

In [7]:
df['status'].unique()

array(['Stayed', 'Joined', 'Churned'], dtype=object)

In [8]:
df.loc[df['status'] == 'Churned', 'churn_label'] = 'Yes'

In [9]:
df['churn_label'].unique()

array(['No', 'Yes'], dtype=object)

In [10]:
df.set_index(['customer_id', 'account_id'], inplace=True)
df['churn_label'] = df['churn_label'].map({'Yes': 1, 'No': 0})

In [11]:
df.dtypes

zip_code                           int64
tenure_months                      int64
num_referrals                      int64
has_internet_service              object
internet_type                     object
has_unlimited_data                object
has_phone_service                 object
has_multiple_lines                object
has_premium_tech_support          object
has_online_security               object
has_online_backup                 object
has_device_protection             object
contract_type                     object
paperless_billing                 object
payment_method                    object
avg_long_distance_fee_monthly    float64
total_long_distance_fee          float64
avg_gb_download_monthly            int64
stream_tv                         object
stream_movie                      object
stream_music                      object
total_monthly_fee                float64
total_charges_quarter            float64
total_refunds                    float64
gender          

In [12]:
df.columns

Index(['zip_code', 'tenure_months', 'num_referrals', 'has_internet_service',
       'internet_type', 'has_unlimited_data', 'has_phone_service',
       'has_multiple_lines', 'has_premium_tech_support', 'has_online_security',
       'has_online_backup', 'has_device_protection', 'contract_type',
       'paperless_billing', 'payment_method', 'avg_long_distance_fee_monthly',
       'total_long_distance_fee', 'avg_gb_download_monthly', 'stream_tv',
       'stream_movie', 'stream_music', 'total_monthly_fee',
       'total_charges_quarter', 'total_refunds', 'gender', 'age',
       'senior_citizen', 'married', 'num_dependents', 'area_id', 'city',
       'latitutde', 'longitude', 'population', 'status', 'churn_label',
       'churn_category', 'churn_reason'],
      dtype='object')

4. EDA

In [13]:
percentage_df = df.groupby(['tenure_months', 'churn_label']).size().reset_index(name='count')
percentage_df['percentage'] = percentage_df.groupby('tenure_months')['count'].transform(lambda x: x / x.sum() * 100)
fig = px.bar(
    data_frame=percentage_df,
    x='tenure_months',
    y='percentage',
    color='churn_label',
    barmode='group',
    title='Relative Percentage of Churn by Tenure',
    labels={'tenure_months': 'Tenure (Months)', 'percentage': 'Percentage', 'churn_label': 'Churn Status'},
    color_discrete_sequence=px.colors.qualitative.Set1
)

fig.show()

In [ ]:
# observation above: there is a higher chance of churning when tenure_months is shorter

In [14]:
bins = [0, 20, 40, 60, 80, 100, 120]
labels = ['0-20', '21-40', '41-60', '61-80', '81-100', '101-120']

tempdf = df.copy()
tempdf['fee_bins'] = pd.cut(tempdf['total_monthly_fee'], bins=bins, labels=labels, right=False)

percentage_df = tempdf.groupby(['fee_bins', 'churn_label']).size().reset_index(name='count')

percentage_df['percentage'] = percentage_df.groupby('fee_bins')['count'].transform(lambda x: x / x.sum() * 100)

fig = px.bar(
    data_frame=percentage_df,
    x='fee_bins',
    y='percentage',
    color='churn_label',
    barmode='group',
    title='Relative Percentage of Churn by Total Monthly Fee',
    labels={'fee_bins': 'Total Monthly Fee', 'percentage': 'Percentage', 'churn_label': 'Churn Status'},
    color_discrete_sequence=px.colors.qualitative.Set1
)

fig.show()

C:\Users\yang_\AppData\Local\Temp\ipykernel_24748\1786357426.py:7: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\yang_\AppData\Local\Temp\ipykernel_24748\1786357426.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [ ]:
# observation above: there is relatively higher chance of churning when total monthly fee is higher

5. feature selection, train test split and label encoding

In [15]:
X = df.drop(columns=['status','churn_label','churn_category','churn_reason'])
y = df['churn_label']

feature_selection = ['tenure_months','total_monthly_fee',
                     'has_internet_service',
                     'contract_type', 
                     'num_referrals',
                     'num_dependents',
                     'married',
                     'has_online_security',
                     'stream_movie',
                     'stream_music',
                     'age',
]
X = X[feature_selection]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

encode_feat = list(X_train.select_dtypes(include=['object']).columns)

label_encoders = {}

X_train_encoded = X_train.copy()
for col in encode_feat:
    le = LabelEncoder()
    X_train_encoded[col] = le.fit_transform(X_train_encoded[col])
    label_encoders[col] = le

X_test_encoded = X_test.copy()
for col in encode_feat:
    X_test_encoded[col] = label_encoders[col].transform(X_test_encoded[col])

In [16]:
encode_feat

['has_internet_service',
 'contract_type',
 'married',
 'has_online_security',
 'stream_movie',
 'stream_music']

In [17]:
X_train['has_internet_service'].value_counts()

has_internet_service
Yes    4410
No     1224
Name: count, dtype: int64

In [18]:
X_train_encoded['has_internet_service'].value_counts()

has_internet_service
1    4410
0    1224
Name: count, dtype: int64

In [19]:
X_test['has_internet_service'].value_counts()

has_internet_service
Yes    1107
No      302
Name: count, dtype: int64

In [20]:
X_test_encoded['has_internet_service'].value_counts()

has_internet_service
1    1107
0     302
Name: count, dtype: int64

In [21]:
X_train['contract_type'].value_counts()

contract_type
Month-to-Month    2896
Two Year          1491
One Year          1247
Name: count, dtype: int64

In [22]:
X_train_encoded['contract_type'].value_counts()

contract_type
0    2896
2    1491
1    1247
Name: count, dtype: int64

In [23]:
X_test['contract_type'].value_counts()

contract_type
Month-to-Month    714
Two Year          392
One Year          303
Name: count, dtype: int64

In [24]:
X_test_encoded['contract_type'].value_counts()

contract_type
0    714
2    392
1    303
Name: count, dtype: int64

In [24]:
X['married'].unique()

array(['Yes', 'No'], dtype=object)

In [25]:
X['has_online_security'].unique()

array(['Yes', 'No'], dtype=object)

In [26]:
X['stream_movie'].unique()

array(['No', 'Yes'], dtype=object)

In [27]:
X['stream_music'].unique()

array(['No', 'Yes'], dtype=object)

In [28]:
with open('label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)

6. training models and hyperparameter tuning

In [29]:
xgbc_model = XGBClassifier(
             objective='binary:logistic',  
             eval_metric='auc',
             tree_method='hist',        
             enable_categorical=True       
)

xgbc_model.fit(X_train_encoded, y_train)

xgbc_y_pred = xgbc_model.predict(X_test_encoded)

xgbc_y_pred_proba = xgbc_model.predict_proba(X_test_encoded)[:, 1]

xgbc_accuracy = accuracy_score(y_test, xgbc_y_pred)
xgbc_auc_score = roc_auc_score(y_test, xgbc_y_pred_proba)

print(f"Accuracy: {xgbc_accuracy:.2f}")
print(f"AUC score: {xgbc_auc_score:.2f}")


Accuracy: 0.84
AUC score: 0.90


In [30]:
feature_names = X_train_encoded.columns

importances = xgbc_model.feature_importances_

feature_importances_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
})

feature_importances_df = feature_importances_df.sort_values(by='importance', ascending=False)
feature_importances_df

,feature,importance
3,contract_type,0.460905
2,has_internet_service,0.146663
4,num_referrals,0.089523
5,num_dependents,0.073480
9,stream_music,0.043572
6,married,0.041281
7,has_online_security,0.034902
8,stream_movie,0.031738
10,age,0.027389
0,tenure_months,0.025675


In [31]:
list(feature_importances_df['feature'])

['contract_type',
 'has_internet_service',
 'num_referrals',
 'num_dependents',
 'stream_music',
 'married',
 'has_online_security',
 'stream_movie',
 'age',
 'tenure_months',
 'total_monthly_fee']

In [32]:
cbc_model = CatBoostClassifier(
            iterations=1000,              
            learning_rate=0.1,            
            depth=6,                      
            loss_function='Logloss',      
            eval_metric='AUC',            
            cat_features=encode_feat, 
            verbose=100                   
)

cbc_model.fit(X_train, y_train)

cbc_y_pred = cbc_model.predict(X_test)

cbc_y_pred_proba = cbc_model.predict_proba(X_test_encoded)[:, 1]

cbc_accuracy = accuracy_score(y_test, cbc_y_pred)
cbc_auc_score = roc_auc_score(y_test, cbc_y_pred_proba)

print(f"Accuracy: {cbc_accuracy:.2f}")
print(f"AUC score: {cbc_auc_score:.2f}")


0:	total: 224ms	remaining: 3m 43s
100:	total: 4.39s	remaining: 39.1s
200:	total: 9.1s	remaining: 36.2s
300:	total: 12.3s	remaining: 28.6s
400:	total: 16.7s	remaining: 24.9s
500:	total: 20.8s	remaining: 20.7s
600:	total: 24.6s	remaining: 16.3s
700:	total: 27.9s	remaining: 11.9s
800:	total: 32.5s	remaining: 8.09s
900:	total: 36.1s	remaining: 3.97s
999:	total: 39.9s	remaining: 0us
Accuracy: 0.83
AUC score: 0.81


In [33]:
param_grid = {
    'max_depth': [3, 5, 7],
    # 'learning_rate': [0.3, 0.2, 0.1],
    # 'subsample': [0.5, 0.6, 0.7],
    'reg_alpha': [0.5, 0.7, 0.9],
    # 'reg_lambda': [0.5, 0.7, 0.9],
    'colsample_bytree': [0.5, 0.7, 0.9],
    'gamma': [0, 0.1, 0.9]
}

grid_search = GridSearchCV(xgbc_model, param_grid, cv=3, scoring='roc_auc', verbose=1)

grid_search.fit(X_train_encoded, y_train)

best_params = grid_search.best_params_
print("Best Hyperparameters:")
print(best_params)

Fitting 3 folds for each of 81 candidates, totalling 243 fits
Best Hyperparameters:
{'colsample_bytree': 0.5, 'gamma': 0.9, 'max_depth': 3, 'reg_alpha': 0.9}


In [34]:
best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test_encoded)
y_pred_proba = best_model.predict_proba(X_test_encoded)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
auc_score = roc_auc_score(y_test, y_pred_proba)

print(f"Accuracy: {accuracy:.2f}")
print(f"AUC score: {auc_score:.2f}")

Accuracy: 0.85
AUC score: 0.92


7. saving the selected model as pkl

In [35]:
with open('best_model.pkl', 'wb') as file:
    pickle.dump(best_model, file)

print("Best model saved as 'best_model.pkl'")

Best model saved as 'best_model.pkl'


In [1]:
!pip list

Package                   Version
------------------------- -----------
asttokens                 2.4.1
attrs                     23.2.0
blinker                   1.6.2
Bottleneck                1.3.7
catboost                  1.2.3
click                     8.1.7
colorama                  0.4.6
comm                      0.2.2
contourpy                 1.2.0
cycler                    0.11.0
debugpy                   1.6.7
decorator                 5.1.1
exceptiongroup            1.2.0
executing                 2.0.1
fastjsonschema            2.20.0
Flask                     3.0.3
fonttools                 4.51.0
graphviz                  0.20.1
greenlet                  3.0.1
importlib_metadata        7.1.0
ipykernel                 6.29.4
ipython                   8.25.0
itsdangerous              2.2.0
jedi                      0.19.1
Jinja2                    3.1.4
joblib                    1.4.2
jsonschema                4.22.0
jsonschema-specifications 2023.12.1
jupyter_client     